<a href="https://colab.research.google.com/github/code1508/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import pandas as pd
# import numpy as np
# import re
# import nltk
# nltk.download('punkt_tab')
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score


# # Load dataset (example dataset)
# df = pd.read_csv('symptom_disease_data.csv')

# # Text Preprocessing
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# stop_words = set(stopwords.words('english'))

# def clean_text(text):
#     text = re.sub(r'\W', ' ', text)
#     text = text.lower()
#     words = word_tokenize(text)
#     words = [word for word in words if word not in stop_words]
#     return " ".join(words)

# df['cleaned_symptoms'] = df['symptoms'].apply(clean_text)

# # Convert symptoms into numerical vectors
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df['cleaned_symptoms'])

# # Encode disease labels
# y = df['disease']
# y = pd.factorize(y)[0]

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train model
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# clf.fit(X_train, y_train)

# # Test model
# y_pred = clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Model Accuracy: {accuracy:.2f}')

# # Function to predict disease from symptoms
# def predict_disease(symptom_text):
#     symptom_text = clean_text(symptom_text)
#     vector = vectorizer.transform([symptom_text])
#     prediction = clf.predict(vector)
#     return df['disease'].unique()[prediction[0]]

# joblib.dump(clf, 'ml_model.pkl')
# joblib.dump(vectorizer, 'vectorizer.pkl')

# # Example usage
# user_input = "I have a stomach ache"
# predicted_disease = predict_disease(user_input)
# print(f"Predicted Disease: {predicted_disease}")


import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('symptom_disease_data.csv')

# Preprocess the text data
def clean_text(text):
    import re
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    import nltk

    nltk.download('stopwords')
    nltk.download('punkt')
    stop_words = set(stopwords.words('english'))

    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

df['cleaned_symptoms'] = df['symptoms'].apply(clean_text)

# Convert text data into vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_symptoms'])

# Encode labels
y = pd.factorize(df['disease'])[0]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train ML model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.2f}')

# Save model and vectorizer
joblib.dump(clf, 'ml_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

print("Model and vectorizer saved!")

user_input = "I have a stomach ache"
predicted_disease = predict_disease(user_input)
print(f"Predicted Disease: {predicted_disease}")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Pa

Model Accuracy: 0.00
Model and vectorizer saved!
Predicted Disease: Measles


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


INSTALLING FLASK


In [4]:
!pip install flask


In [5]:

from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load model and vectorizer
model = joblib.load('ml_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

@app.route('/')
def home():
    return "ML Model API is running!"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        symptoms = data['symptoms']

        # Preprocess input
        def clean_text(text):
            import re
            from nltk.tokenize import word_tokenize
            from nltk.corpus import stopwords
            import nltk

            nltk.download('stopwords')
            nltk.download('punkt')
            stop_words = set(stopwords.words('english'))

            text = re.sub(r'\W', ' ', text)
            text = text.lower()
            words = word_tokenize(text)
            words = [word for word in words if word not in stop_words]
            return " ".join(words)

        cleaned_symptoms = clean_text(symptoms)
        vectorized_input = vectorizer.transform([cleaned_symptoms])

        # Predict
        prediction = model.predict(vectorized_input)
        return jsonify({'predicted_disease': str(prediction[0])})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install twilio
!pip install firebase-admin twilio
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime, timedelta
from twilio.rest import Client

# Initialize Firebase
import firebase_admin
from firebase_admin import credentials

if not firebase_admin._apps:
  cred = credentials.Certificate("firebase.json")
  firebase_admin.initialize_app(cred)
# cred = credentials.Certificate("firebase_key.json")
# firebase_admin.initialize_app(cred)
db = firestore.client()


# Add user medication schedule
def add_medication(user_id, medicine_name, dosage, time):
    doc_ref = db.collection("medications").document(user_id)
    doc_ref.set({
        "medicine_name": medicine_name,
        "dosage": dosage,
        "time": time,
        "status": "scheduled"
    })
    print("Medication schedule saved.")

# Send SMS Reminder

import requests

def send_push_notification(fcm_token, medicine_name):
    url = "https://fcm.googleapis.com/fcm/send"
    headers = {
        "Authorization": "key=YOUR_SERVER_KEY",
        "Content-Type": "application/json"
    }
    payload = {
        "to": fcm_token,
        "notification": {
            "title": "Medication Reminder",
            "body": f"Reminder: Take your medicine {medicine_name} now."
        }
    }

    response = requests.post(url, json=payload, headers=headers)
    print("Push Notification Sent!")

# Example Usage

add_medication("aastha", "Paracetamol", "500mg", "11:37 PM")
send_push_notification("USER_FCM_TOKEN", "Paracetamol")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.0 MB/s eta 0:00:00
Medication schedule saved.
Push Notification Sent!
